<a href="https://colab.research.google.com/github/KeneKing12/Kenechukwu/blob/main/pfa600.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import pandas as pd
from pymodbus.client import ModbusTcpClient
from pymodbus.exceptions import ModbusException
import time

# Connect to Modbus client
client = ModbusTcpClient('127.0.0.1')

# Read input function
def read_input(address):
    result = client.read_holding_registers(address, 1)
    return result.registers[0]

# Write output function
def write_output(address, value):
    result = client.write_register(address, value)
    print(f"Successfully wrote {value} to address {address}")

# Function to control the crane according to JSON instructions
def execute_commands_from_json(json_file):
    with open(json_file, 'r') as file:
        data = json.load(file)
    df = pd.DataFrame(data['actions'])

    for _, action in df.iterrows():
        # Check for vacuum control
        if 'vacuum' in action and not pd.isna(action['vacuum']):
            write_output(3, int(action['vacuum']))  # Turn vacuum on/off

        # Check and set X and Y positions
        if 'setX' in action and not pd.isna(action['setX']) and 'setY' in action and not pd.isna(action['setY']):
            x, y = int(action['setX']), int(action['setY'])
            write_output(1, x)  # Set X position
            write_output(2, y)  # Set Y position

            # Wait until the crane reaches the target position
            while read_input(15) != x or read_input(16) != y:
                time.sleep(0.1)  # Small delay to avoid unnecessary CPU usage


if __name__ == "__main__":
    client.connect()
    json_file = 'crane_commands.json'
    execute_commands_from_json(json_file)
    client.close()


ModuleNotFoundError: No module named 'pymodbus'

In [ ]:

  "actions": [
    {"vacuum": 0},
    {"setX": 55, "setY": 200},
    {"setX": 55, "setY": 82},
    {"vacuum": 1},
    {"setX": 55, "setY": 200},
    {"setX": 945, "setY": 200},
    {"setX": 945, "setY": 82},
    {"vacuum": 0},
    {"setX": 945, "setY": 200}
  ]
}


SyntaxError: unmatched '}' (<ipython-input-2-d6ca95f216a8>, line 12)